In [ ]:
%pip install ../requirements.txt

In [1]:
import cv2
from enum import Enum
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import json
import os


fx=2304.54786556982
fy=2305.875668062
cx=1686.23787612802
cy=1354.98486439791

camera_matrix = np.array([[fx, 0,  cx],
                          [0,  fy, cy],
                          [0,  0,  1 ]], dtype=np.float32)

dist_coeffs = np.zeros((1,4))

with open("../mesh/sedan_keypts.txt") as file:
    keypoints_list = []
    
    for line in file.readlines():
        line = line.strip('\n')
        x,y,z = line.split()
        keypoints_list.append([float(x),float(y),float(z)])
    sedan_keypoints_3D = np.array(keypoints_list,dtype="float")
    

with open("../mesh/sedan.json") as json_file:
    car_model = json.load(json_file)
    sedan_vertices = np.array(car_model['vertices'])  # x: w y: h z: l
    sedan_triangles = np.array(car_model['faces']) - 1

with open("../mesh/suv_keypts.txt") as file:
    keypoints_list = []
    
    for line in file.readlines():
        line = line.strip('\n')
        x,y,z = line.split()
        keypoints_list.append([float(x),float(y),float(z)])
    suv_keypoints_3D = np.array(keypoints_list,dtype="float")
    # suv_keypoints_3D[:,0] = suv_keypoints_3D[:,0] * -1
    # suv_keypoints_3D[:,1] = suv_keypoints_3D[:,1] * -1

with open("../mesh/suv.json") as json_file:
    car_model = json.load(json_file)
    suv_vertices = np.array(car_model['vertices'])  # x: w y: h z: l
    suv_triangles = np.array(car_model['faces']) - 1

    # suv_vertices[:,0] = suv_vertices[:,0] * -1
    # suv_vertices[:,1] = suv_vertices[:,1] * -1

def mask_to_polygons(mask):
    # cv2.RETR_CCOMP flag retrieves all the contours and arranges them to a 2-level
    # hierarchy. External contours (boundary) of the object are placed in hierarchy-1.
    # Internal contours (holes) are placed in hierarchy-2.
    # cv2.CHAIN_APPROX_NONE flag gets vertices of polygons from contours.
    mask = np.ascontiguousarray(mask)  # some versions of cv2 does not support incontiguous arr
    res = cv2.findContours(mask.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    hierarchy = res[-1]
    if hierarchy is None:  # empty mask
        return [], False
    has_holes = (hierarchy.reshape(-1, 4)[:, 3] >= 0).sum() > 0
    res = res[-2]
    res = [x.flatten() for x in res]
    # These coordinates from OpenCV are integers in range [0, W-1 or H-1].
    # We add 0.5 to turn them into real-value coordinate space. A better solution
    # would be to first +0.5 and then dilate the returned polygon by 0.5.
    res = [x + 0.5 for x in res if len(x) >= 6][0].reshape(-1, 2)
    return res

def create_above_water_mask(detect_mask):
    mask_height, mask_wdith = detect_mask.shape

    above_water_mask = np.zeros_like(detect_mask)

    left_to_car = True
    right_to_car = False

    for v in range(0,mask_wdith):
        current_column = detect_mask[:,v]

        if left_to_car:
            if current_column.max() == 1.0:
                left_to_car = False
                vehicle_range = np.where(current_column==1)[0]
                min_u = vehicle_range.min()
                max_u = vehicle_range.max()
                edge_u = max_u
                above_water_mask[0:edge_u+1,0:v] = 1
                above_water_mask[0:min_u,v] = 1
        else:
            if right_to_car:
                above_water_mask[0:edge_u+1,v:mask_wdith] = 1
                break
            else:
                if current_column.max() == 1.0:
                    vehicle_range = np.where(current_column==1)[0]
                    min_u = vehicle_range.min()
                    max_u = vehicle_range.max()
                    edge_u = max_u
                    above_water_mask[0:min_u,v] = 1
                else:
                    right_to_car = True
                    above_water_mask[0:edge_u+1,v:mask_wdith] = 1
    return above_water_mask

def calculate_ground_truth_iou(detect_box,ground_truth_box):
    xmin1, ymin1, xmax1, ymax1 = detect_box
    xmin2, ymin2, xmax2, ymax2 = ground_truth_box
    x_overlap = max(0, min(xmax1, xmax2) - max(xmin1, xmin2))
    y_overlap = max(0, min(ymax1, ymax2) - max(ymin1, ymin2))
    intersection = x_overlap * y_overlap
    union = (xmax1 - xmin1) * (ymax1 - ymin1) + (xmax2 - xmin2) * (ymax2 - ymin2) - intersection
    return float(intersection) / union

def draw_obj(image, vertices, triangles, color):
    for t in triangles:
        coord = np.array(
            [vertices[t[0]][:2], vertices[t[1]][:2],
                vertices[t[2]][:2]], dtype=np.int32
        )
        cv2.polylines(image, np.int32([coord]), 1, color)
        
def draw_obj_mask(image, vertices, triangles, color):
    for t in triangles:
        coord = np.array(
            [vertices[t[0]][:2], vertices[t[1]][:2],
                vertices[t[2]][:2]], dtype=np.int32
        )
        cv2.fillPoly(image, np.int32([coord]),  color)

def draw_mask(image, vertices, triangles, block_vertices_list=[]):
    for t in triangles:
        coord = np.array(
            [vertices[t[0]][:2], vertices[t[1]][:2],
                vertices[t[2]][:2]], dtype=np.int32
        )
        cv2.fillPoly(image, np.int32([coord]), 1)

def water_rising(vertices,index):
    top = np.max(vertices,0)[1]
    bottom = np.min(vertices,0)[1]

    water_level_ratio = index / 100  
    water_level = water_level_ratio * (top - bottom)

    level_bottom = bottom + water_level
    new_vertices = vertices.copy() # x: w y: h z: l
    x_column = new_vertices[:,0]
    y_column = new_vertices[:,1]
    z_column = new_vertices[:,2]
    x_min = x_column[y_column == top].min()
    x_max = x_column[y_column == top].max()
    z_min = z_column[y_column == top].min()
    z_max = z_column[y_column == top].max()
    for i_v, v in enumerate(new_vertices):
        if v[1] >= top - water_level:
            new_vertices[i_v][1] = top - water_level
            new_vertices[i_v][0] = v[0]
            new_vertices[i_v][2] = v[2]
        else:
            new_vertices[i_v] = v

    return new_vertices,water_level

In [2]:
def decide_car_type(keypoint,keypoint_id,keypoints_3D,flag = cv2.SOLVEPNP_EPNP):
    imgPoints = keypoint
    objPoints = keypoints_3D[keypoint_id]

    (return_value, rotation_vector, translation_vector) = cv2.solvePnP(objPoints,
                                                                    imgPoints,
                                                                    camera_matrix,
                                                                    dist_coeffs,
                                                                    flags=flag)

    (projected_keypoints, jacobian_model) = cv2.projectPoints(objPoints, 
                                                                rotation_vector, 
                                                                translation_vector,
                                                                camera_matrix, 
                                                                dist_coeffs)
    projected_keypoints = np.squeeze(projected_keypoints,axis=1)

    reprojectionError = cv2.norm(imgPoints,projected_keypoints,cv2.NORM_L2) / len(imgPoints)
    r_var = np.var(imgPoints - projected_keypoints)
    
    return reprojectionError,(rotation_vector, translation_vector)
    
def draw_result_image(np_image,
                    image_save_name,
                    keypoint,
                    keypoint_id,
                    projected_vertices,
                    projected_keypoints,
                    projected_keypoints_all,
                    right_waterlevel_mask,
                    right_projected_waterlevel_vertices,
                    need_detect_keypoint = False,
                    need_project_keypoint = False,
                    need_project_keypoint_all = False,
                    need_detect_mask = True,
                    need_waterlevel_mask = False,
                    need_waterlevel_obj = False,
                    need_box = False,
                    need_project_obj_mask = False,
                    need_project_obj = False):
    fig = plt.figure()
    ax = fig.gca()
    plt.axis('off')

    copy_image = np_image.copy()

    projected_mask = np.zeros_like(mask)
    draw_mask(projected_mask,projected_vertices,faces)

    if need_detect_keypoint:
        for kpt,kpt_id in zip(keypoint,keypoint_id):
            plt.scatter(int(kpt[0]),int(kpt[1]),c='coral', s=10)
            plt.text(int(kpt[0]), int(kpt[1]), str(kpt_id),color = "blueviolet",fontsize=10)

    if need_project_keypoint:
        for keypoint,kpt_id in zip(projected_keypoints,keypoint_id):
            kpt = keypoint
            plt.scatter(int(kpt[0]),int(kpt[1]),c='blue', s=10)
            plt.text(int(kpt[0]), int(kpt[1]), str(kpt_id),color = "red",fontsize=10)

    if need_project_keypoint_all:
        for i,keypoint in enumerate(projected_keypoints_all):
            kpt = keypoint
            plt.scatter(int(kpt[0]),int(kpt[1]),c='blue', s=10)
            plt.text(int(kpt[0]), int(kpt[1]), str(i),color = "red",fontsize=10)

    if need_project_obj:
        draw_obj(copy_image,projected_vertices,faces,(190,80,50))

    if need_project_obj_mask:
        polygon_seg = patches.Polygon(
            mask_to_polygons(projected_mask),
            fill=True,
            facecolor="lightgreen",
            edgecolor="lightyellow",
            linewidth=1,
            alpha=0.6
        )
        ax.add_patch(polygon_seg)

    if need_detect_mask:
        polygon_seg_2 = patches.Polygon(
            mask_to_polygons(mask),
            fill=True,
            facecolor="salmon",
            edgecolor="lightyellow",
            linewidth=1,
            alpha=0.4
        )
        ax.add_patch(polygon_seg_2)
    
    if need_waterlevel_mask:
        copy_image[right_waterlevel_mask.astype(bool)] = [90,50,255]

    if need_waterlevel_obj:
        draw_obj(copy_image,right_projected_waterlevel_vertices,faces,(60,120,230))

    plt.imshow(copy_image) 
    plt.savefig(image_save_name,bbox_inches='tight',dpi=200,pad_inches=0.0)
    plt.close()


In [4]:
image_folder = "../images/Flood/"
keypoint_result_folder = "../result_keypoint/"
mask_result_folder = "../result_mask/"
project_result_folder = "../result_project/"

file_names = os.listdir(image_folder)
for image_name in file_names:
    
    base_name = os.path.splitext(image_name)[0]

    single_folder = project_result_folder+base_name+"/"
    if not os.path.exists(single_folder):
        os.makedirs(single_folder)

    label_name = keypoint_result_folder+base_name+"/"+base_name+".json"

    text_record = open(single_folder+base_name+".txt","w")

    pil_image = Image.open(image_folder+image_name)
    im_width, im_height = pil_image.size
    np_image = np.array(pil_image)

    with open(mask_result_folder+label_name) as json_file:
        data = json.load(json_file)
        cars_data = data["cars"]

    waterlevel_data = {"cars":[]}
    for car_id,car in enumerate(cars_data):
        text_record.write(f"car {car_id}\n")

        box = car['box']
        mask = np.array(car['mask'])
        keypoint = np.array(car['keypoint'])
        keypoint_id = np.array(car['keypoint_id'])

        if len(keypoint) <= 3:
            continue
        
        if len(keypoint) >= 6:
            r_error_sedan_epnp,trans_sedan_epnp = decide_car_type(keypoint,keypoint_id,sedan_keypoints_3D,cv2.SOLVEPNP_EPNP)
            r_error_sedan_iter,trans_sedan_iter = decide_car_type(keypoint,keypoint_id,sedan_keypoints_3D,cv2.SOLVEPNP_ITERATIVE)

            text_record.write(f"r error sedan epnp: {r_error_sedan_epnp}\n")
            text_record.write(f"r error sedan iter: {r_error_sedan_iter}\n")

            if r_error_sedan_epnp < r_error_sedan_iter:
                r_error_sedan = r_error_sedan_epnp
                trans_sedan = trans_sedan_epnp
            else:
                r_error_sedan = r_error_sedan_iter
                trans_sedan = trans_sedan_iter

            r_error_suv_epnp,trans_suv_epnp = decide_car_type(keypoint,keypoint_id,suv_keypoints_3D,cv2.SOLVEPNP_EPNP)
            r_error_suv_iter,trans_suv_iter = decide_car_type(keypoint,keypoint_id,suv_keypoints_3D,cv2.SOLVEPNP_ITERATIVE)

            text_record.write(f"r error suv epnp: {r_error_suv_epnp}\n")
            text_record.write(f"r error suv iter: {r_error_suv_iter}\n")

            if r_error_suv_epnp < r_error_suv_iter:
                r_error_suv = r_error_suv_epnp
                trans_suv = trans_suv_epnp
            else:
                r_error_suv = r_error_suv_iter
                trans_suv = trans_suv_iter

        else:
            r_error_sedan,trans_sedan = decide_car_type(keypoint,keypoint_id,sedan_keypoints_3D,cv2.SOLVEPNP_EPNP)
            r_error_suv,trans_suv = decide_car_type(keypoint,keypoint_id,suv_keypoints_3D,cv2.SOLVEPNP_EPNP)

        
        car_type = ""
        if r_error_sedan < r_error_suv:
            car_type = "sedan"
            text_record.write("sedan\n")
            keypoints_3D = sedan_keypoints_3D.copy()
            vertices = sedan_vertices.copy()
            faces = sedan_triangles.copy()
            rotation_vector,translation_vector = trans_sedan
            r_error = r_error_sedan
        else:
            car_type = "suv"
            text_record.write("suv\n")
            keypoints_3D = suv_keypoints_3D.copy()
            vertices = suv_vertices.copy()
            faces = suv_triangles.copy()
            rotation_vector,translation_vector = trans_suv
            r_error = r_error_suv

        (projected_vertices, jacobian_model) = cv2.projectPoints(vertices, 
                                                            rotation_vector, 
                                                            translation_vector,
                                                            camera_matrix, 
                                                            dist_coeffs)
        projected_vertices = np.squeeze(projected_vertices,axis=1)

        keypoints_3D_visable = keypoints_3D[keypoint_id]
        (projected_keypoints, jacobian_model) = cv2.projectPoints(keypoints_3D_visable, 
                                                                    rotation_vector, 
                                                                    translation_vector,
                                                                    camera_matrix, 
                                                                    dist_coeffs)
        projected_keypoints = np.squeeze(projected_keypoints,axis=1)

        (projected_keypoints_all, jacobian_model) = cv2.projectPoints(keypoints_3D, 
                                                                    rotation_vector, 
                                                                    translation_vector,
                                                                    camera_matrix, 
                                                                    dist_coeffs)
        projected_keypoints_all = np.squeeze(projected_keypoints_all,axis=1)

        max_iou_waterlevel = 0
        right_waterlevel = 0
        right_waterlevel_mask = 0
        right_projected_waterlevel_vertices = 0
        all_ious = []
        for i in range(0,100):
            filtered_vertices,water_level = water_rising(vertices,i)
            (projected_waterlevel_vertices, _) = cv2.projectPoints(filtered_vertices, 
                                                    rotation_vector, 
                                                    translation_vector,
                                                    camera_matrix, 
                                                    dist_coeffs)
            projected_waterlevel_vertices = np.squeeze(projected_waterlevel_vertices,axis=1)
            waterlevel_mask = np.zeros_like(mask)
            draw_mask(waterlevel_mask,projected_waterlevel_vertices,faces)

            overlap_waterlevel_mask = waterlevel_mask + mask
            area_detected = mask.sum()
            area_waterlevel = waterlevel_mask.sum()
            area_overlap_waterlevel = (overlap_waterlevel_mask==2).sum()
            area_union_waterlevel = area_detected + area_waterlevel - area_overlap_waterlevel

            iou_waterlevel = area_overlap_waterlevel / area_union_waterlevel
            all_ious.append(iou_waterlevel)
            if iou_waterlevel > max_iou_waterlevel:
                max_iou_waterlevel = iou_waterlevel
                right_waterlevel = water_level
                right_waterlevel_mask = waterlevel_mask
                right_projected_waterlevel_vertices = projected_waterlevel_vertices

        text_record.write(f"waterlevel: {round(right_waterlevel,2)}\n")

        waterlevel_data["cars"].append({
            "box":box,
            "mask":mask.tolist(),
            "keypoint":keypoint.tolist(),
            "keypoint_id":keypoint_id.tolist(),
            "r_vec":rotation_vector.tolist(),
            "t_vec":translation_vector.tolist(),
            "reprojection_error":r_error,
            "type":car_type,
            "water_level":round(right_waterlevel,2)
        })

        draw_result_image(np_image,
                        single_folder+base_name+f"_car_{car_id}_object_project.jpg",
                        keypoint,
                        keypoint_id,
                        projected_vertices,
                        projected_keypoints,
                        projected_keypoints_all,
                        right_waterlevel_mask,
                        right_projected_waterlevel_vertices,
                        need_detect_keypoint = False,
                        need_project_keypoint = False,
                        need_project_keypoint_all = False,
                        need_detect_mask = True,
                        need_waterlevel_mask = False,
                        need_box = False,
                        need_project_obj_mask = False,
                        need_project_obj = True)
        draw_result_image(np_image,
                        single_folder+base_name+f"_car_{car_id}_water_level_mask.jpg",
                        keypoint,
                        keypoint_id,
                        projected_vertices,
                        projected_keypoints,
                        projected_keypoints_all,
                        right_waterlevel_mask,
                        right_projected_waterlevel_vertices,
                        need_detect_keypoint = False,
                        need_project_keypoint = False,
                        need_project_keypoint_all = False,
                        need_detect_mask = True,
                        need_waterlevel_mask = True,
                        need_box = False,
                        need_project_obj_mask = False,
                        need_project_obj = False)
        draw_result_image(np_image,
                        single_folder+base_name+f"_car_{car_id}_water_level_object.jpg",
                        keypoint,
                        keypoint_id,
                        projected_vertices,
                        projected_keypoints,
                        projected_keypoints_all,
                        right_waterlevel_mask,
                        right_projected_waterlevel_vertices,
                        need_detect_keypoint = False,
                        need_project_keypoint = False,
                        need_project_keypoint_all = False,
                        need_detect_mask = True,
                        need_waterlevel_mask = False,
                        need_waterlevel_obj = True,
                        need_box = False,
                        need_project_obj_mask = False,
                        need_project_obj = False)
        draw_result_image(np_image,
                        single_folder+base_name+f"_car_{car_id}_keypoint.jpg",
                        keypoint,
                        keypoint_id,
                        projected_vertices,
                        projected_keypoints,
                        projected_keypoints_all,
                        right_waterlevel_mask,
                        right_projected_waterlevel_vertices,
                        need_detect_keypoint = True,
                        need_project_keypoint = True,
                        need_project_keypoint_all = False,
                        need_detect_mask = True,
                        need_waterlevel_mask = False,
                        need_box = False,
                        need_project_obj_mask = True,
                        need_project_obj = False)
    
    with open(single_folder+base_name+".json","w") as file:
        file.write(json.dumps(waterlevel_data))

In [30]:
image_folder = "../images/Flood/"
keypoint_result_folder = "../result_keypoint/"
mask_result_folder = "../result_mask/"
project_result_folder = "../result_project/"

image_name = "Flood_90.jpg"

base_name = os.path.splitext(image_name)[0]

single_folder = project_result_folder+base_name+"/"
if not os.path.exists(single_folder):
    os.makedirs(single_folder)

label_name = keypoint_result_folder+base_name+"/"+base_name+".json"

text_record = open(single_folder+base_name+".txt","w")

pil_image = Image.open(image_folder+image_name)
im_width, im_height = pil_image.size
np_image = np.array(pil_image)

with open(mask_result_folder+label_name) as json_file:
    data = json.load(json_file)
    cars_data = data["cars"]

waterlevel_data = {"cars":[]}
for car_id,car in enumerate(cars_data):
    
    print(f"car {car_id}")

    box = car['box']
    mask = np.array(car['mask'])
    keypoint = np.array(car['keypoint'])
    keypoint_id = np.array(car['keypoint_id'])

    
    r_error_sedan_epnp,trans_sedan_epnp = decide_car_type(keypoint,keypoint_id,sedan_keypoints_3D,cv2.SOLVEPNP_EPNP)

    r_error_sedan_iter,trans_sedan_iter = decide_car_type(keypoint,keypoint_id,sedan_keypoints_3D,cv2.SOLVEPNP_ITERATIVE)

    print(f"r error sedan epnp: {r_error_sedan_epnp}")
    print(f"r error sedan iter: {r_error_sedan_iter}")

    if r_error_sedan_epnp < r_error_sedan_iter:
        r_error_sedan = r_error_sedan_epnp
        trans_sedan = trans_sedan_epnp
    else:
        r_error_sedan = r_error_sedan_iter
        trans_sedan = trans_sedan_iter

    r_error_suv_epnp,trans_suv_epnp = decide_car_type(keypoint,keypoint_id,suv_keypoints_3D,cv2.SOLVEPNP_EPNP)
    r_error_suv_iter,trans_suv_iter = decide_car_type(keypoint,keypoint_id,suv_keypoints_3D,cv2.SOLVEPNP_ITERATIVE)

    print(f"r error suv epnp: {r_error_suv_epnp}")
    print(f"r error suv iter: {r_error_suv_iter}")

    if r_error_suv_epnp < r_error_suv_iter:
        r_error_suv = r_error_suv_epnp
        trans_suv = trans_suv_epnp
    else:
        r_error_suv = r_error_suv_iter
        trans_suv = trans_suv_iter

    if r_error_sedan < r_error_suv:
        car_type = "sedan"
        keypoints_3D = sedan_keypoints_3D.copy()
        vertices = sedan_vertices.copy()
        faces = sedan_triangles.copy()
        rotation_vector,translation_vector = trans_sedan
        r_error = r_error_sedan
    else:
        car_type = "suv"
        keypoints_3D = suv_keypoints_3D.copy()
        vertices = suv_vertices.copy()
        faces = suv_triangles.copy()
        rotation_vector,translation_vector = trans_suv
        r_error = r_error_suv

    (projected_vertices, jacobian_model) = cv2.projectPoints(vertices, 
                                                                rotation_vector, 
                                                                translation_vector,
                                                                camera_matrix, 
                                                                dist_coeffs)
    projected_vertices = np.squeeze(projected_vertices,axis=1)

    keypoints_3D_visable = keypoints_3D[keypoint_id]
    (projected_keypoints, jacobian_model) = cv2.projectPoints(keypoints_3D_visable, 
                                                                rotation_vector, 
                                                                translation_vector,
                                                                camera_matrix, 
                                                                dist_coeffs)
    projected_keypoints = np.squeeze(projected_keypoints,axis=1)

    (projected_keypoints_all, jacobian_model) = cv2.projectPoints(keypoints_3D, 
                                                                rotation_vector, 
                                                                translation_vector,
                                                                camera_matrix, 
                                                                dist_coeffs)
    projected_keypoints_all = np.squeeze(projected_keypoints_all,axis=1)

    max_iou_waterlevel = 0
    right_waterlevel = 0
    right_waterlevel_mask = 0
    right_projected_waterlevel_vertices = 0
    all_ious = []
    for i in range(0,100):
        filtered_vertices,water_level = water_rising(vertices,i)
        (projected_waterlevel_vertices, _) = cv2.projectPoints(filtered_vertices, 
                                                rotation_vector, 
                                                translation_vector,
                                                camera_matrix, 
                                                dist_coeffs)
        projected_waterlevel_vertices = np.squeeze(projected_waterlevel_vertices,axis=1)
        waterlevel_mask = np.zeros_like(mask)
        draw_mask(waterlevel_mask,projected_waterlevel_vertices,faces)

        overlap_waterlevel_mask = waterlevel_mask + mask
        area_detected = mask.sum()
        area_waterlevel = waterlevel_mask.sum()
        area_overlap_waterlevel = (overlap_waterlevel_mask==2).sum()
        area_union_waterlevel = area_detected + area_waterlevel - area_overlap_waterlevel

        iou_waterlevel = area_overlap_waterlevel / area_union_waterlevel
        all_ious.append(iou_waterlevel)
        if iou_waterlevel > max_iou_waterlevel:
            max_iou_waterlevel = iou_waterlevel
            right_waterlevel = water_level
            right_waterlevel_mask = waterlevel_mask
            right_projected_waterlevel_vertices = projected_waterlevel_vertices

    waterlevel_data["cars"].append({
        "box":box,
        "mask":mask.tolist(),
        "keypoint":keypoint.tolist(),
        "keypoint_id":keypoint_id.tolist(),
        "r_vec":rotation_vector.tolist(),
        "t_vec":translation_vector.tolist(),
        "reprojection_error":r_error,
        "type":car_type,
        "water_level":round(right_waterlevel,2)
    })

    draw_result_image(np_image,
                    single_folder+base_name+f"_car_{car_id}_object_project.jpg",
                    keypoint,
                    keypoint_id,
                    projected_vertices,
                    projected_keypoints,
                    projected_keypoints_all,
                    right_waterlevel_mask,
                    right_projected_waterlevel_vertices,
                    need_detect_keypoint = False,
                    need_project_keypoint = False,
                    need_project_keypoint_all = False,
                    need_detect_mask = True,
                    need_waterlevel_mask = False,
                    need_box = False,
                    need_project_obj_mask = False,
                    need_project_obj = True)
    draw_result_image(np_image,
                    single_folder+base_name+f"_car_{car_id}_water_level_mask.jpg",
                    keypoint,
                    keypoint_id,
                    projected_vertices,
                    projected_keypoints,
                    projected_keypoints_all,
                    right_waterlevel_mask,
                    right_projected_waterlevel_vertices,
                    need_detect_keypoint = False,
                    need_project_keypoint = False,
                    need_project_keypoint_all = False,
                    need_detect_mask = True,
                    need_waterlevel_mask = True,
                    need_box = False,
                    need_project_obj_mask = False,
                    need_project_obj = False)
    draw_result_image(np_image,
                    single_folder+base_name+f"_car_{car_id}_water_level_object.jpg",
                    keypoint,
                    keypoint_id,
                    projected_vertices,
                    projected_keypoints,
                    projected_keypoints_all,
                    right_waterlevel_mask,
                    right_projected_waterlevel_vertices,
                    need_detect_keypoint = False,
                    need_project_keypoint = False,
                    need_project_keypoint_all = False,
                    need_detect_mask = True,
                    need_waterlevel_mask = False,
                    need_waterlevel_obj = True,
                    need_box = False,
                    need_project_obj_mask = False,
                    need_project_obj = False)
    draw_result_image(np_image,
                    single_folder+base_name+f"_car_{car_id}_keypoint.jpg",
                    keypoint,
                    keypoint_id,
                    projected_vertices,
                    projected_keypoints,
                    projected_keypoints_all,
                    right_waterlevel_mask,
                    right_projected_waterlevel_vertices,
                    need_detect_keypoint = True,
                    need_project_keypoint = True,
                    need_project_keypoint_all = False,
                    need_detect_mask = True,
                    need_waterlevel_mask = False,
                    need_box = False,
                    need_project_obj_mask = True,
                    need_project_obj = False)
with open(single_folder+base_name+".json","w") as file:
        file.write(json.dumps(waterlevel_data))

car 0
r error sedan epnp: 25.796352575127244
r error sedan iter: 12.9850018888077
r error suv epnp: 1.434572604261651
r error suv iter: 8.419774083531026
